In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re

In [ ]:
def read_dataframe(path):
  df=pd.read_excel(path)
  df.drop(['pdf','href formula','href','Case Number','Case Number.1','original order','Unnamed: 21','Unnamed: 22'],axis=1,inplace=True)
  df.rename(columns={'Species ':'Species','Unnamed: 11':'Fatality'},inplace=True)
  df=df.iloc[:6906]

  return df

In [ ]:
df=read_dataframe('/content/drive/My Drive/GSAF5.xls')

In [ ]:
pd.set_option('display.max_columns',None,'display.max_rows',None)

In [ ]:
df.describe()

,Year
count,6904.000000
mean,1933.962920
std,273.577914
min,0.000000
25%,1947.000000
50%,1984.000000
75%,2009.000000
max,2023.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6906 entries, 0 to 6905
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      6906 non-null   object 
 1   Year      6904 non-null   float64
 2   Type      6888 non-null   object 
 3   Country   6856 non-null   object 
 4   State     6425 non-null   object 
 5   Location  6342 non-null   object 
 6   Activity  6322 non-null   object 
 7   Name      6687 non-null   object 
 8   Sex       6327 non-null   object 
 9   Age       3920 non-null   object 
 10  Injury    6871 non-null   object 
 11  Fatality  6344 non-null   object 
 12  Time      3388 non-null   object 
 13  Species   3786 non-null   object 
 14  Source    6887 non-null   object 
dtypes: float64(1), object(14)
memory usage: 809.4+ KB


In [ ]:
df.Sex.unique()

array(['M', 'F', nan, ' M', 'M ', 'lli', 'M x 2', 'N', '.'], dtype=object)

In [ ]:
def clean_sex_column(df):
  pattern= r'.*M.*'
  df.Sex.replace(pattern,'M',regex=True,inplace=True)
  df.Sex.replace('lli','M',inplace=True)
  df.iloc[5531,8]=np.nan
  df.iloc[6728]='M'
  df.Sex[df.Sex=='.']=np.nan
  pattern1=r'&|passengers|crew\b|occupants|males\b|Occupants|people|men\b|children|others?\b|migrants|immigrants|couple\b|refugees\b|slaves|poachers|tourists|swimmers'
  mask=df.Name.str.contains(pattern1,regex=True)
  df.Sex[(mask)&(df.Sex.isnull())]='multiple'

  return df

In [ ]:
df=clean_sex_column(df)
df.Sex.unique()

array(['M', 'F', nan, 'multiple'], dtype=object)

In [ ]:
df[(df.Sex.isnull())]

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatality,Time,Species,Source
14,23 Aug-2023,2023.0,Unprovoked,USA,North Carolina,"Oak Island, Brunswick County",Wading,child,NaN,9,Minor injury to shoulder,N,NaN,NaN,"WCNC, 08/24/2023"
117,19-Jul-2022,2022.0,Questionable,USA,Florida,"Cape Canaveral, Brevard County",NaN,NaN,NaN,NaN,minor injuries,N,10jh45,Shark involvement not confirmed,"WESH, 7/20/2022"
141,27-May-2022,2022.0,Unprovoked,AUSTRALIA,Western Australia,Yallingup,Surfing,NaN,NaN,NaN,Knocked off board by a shark. No injury,N,NaN,"Bronze whaler shark, 2m","Perth Now, 5/27/2022"
145,07-May-2022,2022.0,Unprovoked,FRENCH POLYNESIA,Bora Bora,To'opua,NaN,NaN,NaN,NaN,Hand bitten,N,NaN,NaN,"Franceinfo, 5/10/202232/2021"
257,17-Apr-2021,2021.0,Provoked,USA,North Carolina,Outer Banks,Tagging sharks,fishing boat Sarah Brent,NaN,NaN,"No injury to occupants, shark bit boat after b...",N,NaN,"Mako shark, 4 juvenile","News Observer, 4/29/2021"
278,22-Jan-2021,2021.0,Watercraft,USA,Florida,Off Tampa Bay,Fishing,NaN,NaN,NaN,No injury to occupants,N,NaN,White shark,"J. McCallister, GSAF"
322,30-Aug-2020,2020.0,Watercraft,AUSTRALIA,New South Wales,Forster,Fishing / Filming sharks feeding on whale carcass,Occupant: Dean Butler,NaN,NaN,"Shark bit boat, no injury to occupant",N,Afternoon,White shark,"Nine News, 8/30/2020"
352,20-Jun-2020,2020.0,Unprovoked,BAHAMAS,Exumas,"Pig Beach, Pig Island",NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,"M. Michaelson, GSAF"
363,17 May 2020,2020.0,Unprovoked,FRENCH POLYNESIA,Tahiti,Vallée Blanche,Snorkeling,NaN,NaN,NaN,Leg bitten,N,NaN,Blacktip shark,"K. McMurray, TrackingSharks.com"
396,20-Dec-2019,2019.0,Provoked,AUSTRALIA,New South Wales,Shellharbour,Fishing,NaN,NaN,NaN,PROVOKED INCIDENT,NaN,NaN,White shark,"B. Myatt, GSAF"


In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
# activity

In [ ]:
def give_enumerated_dict_from_list(list_of_words):


  dictionary_of_word_index={}

  for i,word in enumerate(list_of_words):
    dictionary_of_word_index[word]=i

  return dictionary_of_word_index



def remove_stopwords(string):
  if pd.isna(string):
    string=''
  if type(string) != str:
    raise ValueError(f'object is of type {type(string)}, object is {string}')

  tokenized_list=nltk.word_tokenize(string)

  stopwords_dict=give_enumerated_dict_from_list(stopwords.words('english'))
  tokenized_dict=give_enumerated_dict_from_list(tokenized_list)

  no_stopwords_list=[]
  for word in tokenized_dict.keys():
    if not stopwords_dict.get(word,False):
      no_stopwords_list.append(word)


  return no_stopwords_list

def pos_tag_verb_selection(string):
  string=remove_stopwords(string)

  tagged_string=nltk.pos_tag(string)

  list_of_only_verbs=[]

  for word,pos in dict(tagged_string).items():
    if 'V' in pos:
      list_of_only_verbs.append(word)

  list_of_only_lemma=[]

  for word in list_of_only_verbs:
    list_of_only_lemma.append(lemmatizer.lemmatize(word.lower(),'v'))

  return list_of_only_lemma



In [ ]:
df['Activity_tokens']=df.Activity.apply(pos_tag_verb_selection)

In [ ]:
df.Activity_tokens.value_counts().shape

(587,)